**Importing Modules and Reading Dataset**

In [2]:
import pandas as pd

# Reading dataset
df = pd.read_csv("AB_NYC_2019.csv")

#Saving uncleaned dataset
df.to_csv("Uncleaned_Dataset", index=False)

**Overview Information**

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [5]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
count,4.889500e+04,4.889500e+04,48895.000000,48895.000000,48895.000000,48895.000000,48895.000000,38843.000000,48895.000000,48895.000000
mean,1.901714e+07,6.762001e+07,40.728949,-73.952170,152.720687,7.029962,23.274466,1.373221,7.143982,112.781327
std,1.098311e+07,7.861097e+07,0.054530,0.046157,240.154170,20.510550,44.550582,1.680442,32.952519,131.622289
min,2.539000e+03,2.438000e+03,40.499790,-74.244420,0.000000,1.000000,0.000000,0.010000,1.000000,0.000000
25%,9.471945e+06,7.822033e+06,40.690100,-73.983070,69.000000,1.000000,1.000000,0.190000,1.000000,0.000000
50%,1.967728e+07,3.079382e+07,40.723070,-73.955680,106.000000,3.000000,5.000000,0.720000,1.000000,45.000000
75%,2.915218e+07,1.074344e+08,40.763115,-73.936275,175.000000,5.000000,24.000000,2.020000,2.000000,227.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,1250.000000,629.000000,58.500000,327.000000,365.000000


In [6]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

**Dropping and Categorzing Columns**

In [8]:
# Drop nonessential columns
df.drop(columns=["reviews_per_month", "last_review"], inplace=True)

In [9]:
# Categorizing appropriate columns for organization
df["room_type"] = df["room_type"].astype("category")
df["neighbourhood_group"] = df["neighbourhood_group"].astype("category")
df["neighbourhood"] = df["neighbourhood"].astype("category")

**Handling Missing Values and Duplicates**

In [11]:
df.isnull().sum()

id                                 0
name                              16
host_id                            0
host_name                         21
neighbourhood_group                0
neighbourhood                      0
latitude                           0
longitude                          0
room_type                          0
price                              0
minimum_nights                     0
number_of_reviews                  0
calculated_host_listings_count     0
availability_365                   0
dtype: int64

In [12]:
# Drop missing listing/host names since a low amount
df.dropna(subset="name", inplace=True)
df.dropna(subset="host_name", inplace=True)

In [13]:
# Checking if duplicates in unique booking ids
df = df.drop_duplicates(subset=["id"])

In [14]:
# Checking if theres multiple hosts under same the unique host id
unique_host_names = df.groupby("host_id")["host_name"].nunique()
if (unique_host_names > 1).any():
    print("Multiple hosts")
else:
    print("Unique.")

Unique.


In [15]:
df.isnull().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

**Cleaning Host and Listing Names for User Readability**

In [17]:
# Cleaning host names
df["host_name"] = (
    df["host_name"]
    .str.strip()
    .str.title()
    .str.replace(r"[^a-zA-Z0-9 '&-]", '', regex=True)
)

In [18]:
# Cleaning listing names
df["name"] = (
    df["name"]
    .str.strip()
    .str.title()
    .str.replace(r"[^a-zA-Z0-9 '&-]", '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
)

**Setting Boundaries for Prices, Availability and Minimum Nights**

In [20]:
# Removing listings less than $1 and greater than $10,000
df = df[(df["price"] >= 1) & (df["price"] <= 10000)]

In [21]:
# Removing listings with no availability
df = df[(df["availability_365"] >= 1) & (df["availability_365"] <= 365)]

In [22]:
# Removing listings with no minimum nights and minimum nights greater than 365 days
df = df[(df["minimum_nights"] >= 1) & (df["minimum_nights"] <= 365)]

In [23]:
df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
count,3.132900e+04,3.132900e+04,31329.000000,31329.000000,31329.000000,31329.000000,31329.000000,31329.000000,31329.000000
mean,2.101257e+07,8.148038e+07,40.728407,-73.948753,162.122953,8.044368,31.852820,10.342686,175.754413
std,1.148577e+07,8.674994e+07,0.056557,0.051402,254.533776,18.308936,51.642146,40.720912,126.166114
min,2.539000e+03,2.571000e+03,40.499790,-74.244420,10.000000,1.000000,0.000000,1.000000,1.000000
25%,1.169731e+07,8.620509e+06,40.688400,-73.983280,70.000000,2.000000,2.000000,1.000000,55.000000
50%,2.257006e+07,4.109936e+07,40.723720,-73.954120,112.000000,3.000000,10.000000,1.000000,167.000000
75%,3.120348e+07,1.452412e+08,40.763040,-73.930150,189.000000,5.000000,39.000000,3.000000,305.000000
max,3.648724e+07,2.743213e+08,40.913060,-73.712990,10000.000000,365.000000,629.000000,327.000000,365.000000


**Saving Cleaned Dataset**

In [25]:
df.to_csv("Cleaned_Dataset", index=False)